<a href="https://colab.research.google.com/github/LeeDongGeon1996/TensorFlow-study-note/blob/master/Lab_10_2_Weight_Initialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab_10_2 Weight Initialization

 * model의 weight를 He_uniform으로 초기화하였다.


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np


TensorFlow 2.x selected.


In [0]:

def load_mnist():
  (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
  #print(type(train_data[0][0][0]))
  train_data, test_data = normalize(train_data, test_data)
  train_labels = tf.keras.utils.to_categorical(train_labels, 10)
  test_labels = tf.keras.utils.to_categorical(test_labels, 10)

  return train_data, train_labels, test_data, test_labels


def normalize(train_data, test_data):
  train_data = train_data.astype(np.float32) / 255.0
  test_data = test_data.astype(np.float32) / 255.0
  return train_data, test_data


def flatten():
  return tf.keras.layers.Flatten()

def dense(channel, activation_fn='relu', weight_unit='random_uniform'):
  return tf.keras.layers.Dense(units=channel, activation=activation_fn, kernel_initializer=weight_unit)


class mnist_model(tf.keras.Model):
  def __init__(self, label_dim):
    super(mnist_model, self).__init__()

    self.model = tf.keras.Sequential()
    
    #input_layer
    self.model.add(flatten())

    #hidden layer
    self.model.add(dense(1024))
    self.model.add(dense(256))

    #output layer
    self.model.add(dense(label_dim, activation_fn=None))
    #self.model.add(dense(label_dim, activation_fn=tf.keras.activations.softmax))


  def call(self, x, training=None, mask=None):
    x = self.model(x)
    return x

def loss_fn(model, images, labels):
  logits = model(images ,training=True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, logits))
  return loss

def accuracy_fn(model, images, labels):
  logits = model(images, training=False)
  compare = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
  accuracy = tf.reduce_mean(tf.cast(compare, tf.float32))
  return accuracy

#def grad(model, images, labels):

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
def fit(model, dataset, test_data, epoch=5, verbose=1):
  for i in range(epoch):
    for x_train, y_train in dataset:
      with tf.GradientTape() as g:
        loss = loss_fn(model, x_train, y_train)
        grads = g.gradient(loss, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

        for test_x, test_y in test_data:
          acc = accuracy_fn(model, test_x, test_y)
    if (i%verbose)==0:
          print("Epoch: {}, loss: {:.4f}, acc: {:.4f}".format(i, loss, acc))



In [0]:
train_x, train_y, test_x, test_y = load_mnist()
label_dim = 10
network = mnist_model(label_dim)
batch_size=1024

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

fit(network, train_dataset, test_dataset)



Epoch: 0, loss: 1.9003, acc: 0.5784
Epoch: 1, loss: 1.6985, acc: 0.7664
Epoch: 2, loss: 1.6209, acc: 0.8607
Epoch: 3, loss: 1.6162, acc: 0.8585
Epoch: 4, loss: 1.6044, acc: 0.8628
